In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as DT
from matplotlib import style

style.use('fivethirtyeight')

In [40]:
# 1 minute EURUSD prices
## Work
# m1 = pd.read_csv(
#     'C:/Users/kwilliams/Google Drive/dev/python/projects/DochianChannel/Indicator_Anaylsis/EURUSD1_2015.csv',
#     header=None)
## Home
m1 = pd.read_csv(
    'C:/Users/Kevin/Google Drive/dev/python/projects/DochianChannel/Indicator_Anaylsis/EURUSD1_2015.csv',
    header=None)

In [41]:
m1.columns = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Ticks']
m1['DateTime'] = m1['Date'] + ' ' + m1['Time']
m1['DateTime'] = pd.to_datetime(m1['DateTime'],format='%Y.%m.%d %H:%M')
#m1.set_index('DateTime', inplace=True)
#m1.head()

In [42]:
def movingaverage(values,window):
    weigths = np.repeat(1.0, window)/window
    smas = np.convolve(values, weigths, 'valid')
    return smas # as a numpy array


def ExpMovingAverage(values, window):
    weights = np.exp(np.linspace(-1., 0., window))
    weights /= weights.sum()
    a =  np.convolve(values, weights, mode='full')[:len(values)]
    a[:window] = a[window]
    return a


def computeMACD(x, slow=26, fast=12):
    """
    compute the MACD (Moving Average Convergence/Divergence) using a fast and slow exponential moving avg'
    return value is emaslow, emafast, macd which are len(x) arrays
    """
    emaslow = ExpMovingAverage(x, slow)
    emafast = ExpMovingAverage(x, fast)
    return emaslow, emafast, emafast - emaslow

In [43]:
m1['MACD_slow'], m1['MACD_fast'], m1['MACD'] = computeMACD(m1['Close'])

In [44]:
m1['MACD_slow1'] = pd.ewma(m1['Close'], min_periods=26, halflife=13)

In [12]:

#m1.iloc[:,('DateTime')].head(3)
#m1.head()

In [21]:
m1.set_index(['DateTime'], inplace=True)
start = m1.index.searchsorted(DT.datetime(2015, 1, 1))
end = m1.index.searchsorted(DT.datetime(2015, 12, 31))

#m1_2015 = m1['2015']
m1_2015 = m1.ix[start:end]

In [45]:
m1_2015['2015-03'].corr()['Close']['MACD']

0.010456566734769218

In [46]:
trend_labels = ['UP1', 'FLAT1', 'DOWN1', 'UP2', 'DOWN2', 'FLAT2', 'DOWN3', 'UP3', 'DOWN4', 'UP4', 
               'DOWN5', 'UP6', 'FLAT3']
trend_range_DT_sec = pd.Series((1420414890, 1421139880, 1422408640, 1424763719, 1426153060, 1426664604, 1430408260,
                               1431608358, 1433516560, 1434035260, 1439409580, 1443017060, 1447421435, 1450310490))

In [48]:
# function to convert 'DateTime' to seconds
def seconds(datetime):
    return (datetime - DT.datetime(1970, 1, 1)).total_seconds()

m1_2015.reset_index(inplace=True)
#study = df_Ind.loc[:, ('DateTime', 'Trend', 'BBAND(100)', 'MACD(12) MAIN', 'MACD(2) SIGNAL', 'MACD(10) SIGNAL', 'MACD(12) SIGNAL' , 'MACD(20) SIGNAL')]
m1_2015['DateTime_sec'] = m1_2015['DateTime'].apply(seconds)


In [49]:
m1_2015.set_index('DateTime', inplace=True)

In [50]:
m1_2015['Trend'] = pd.cut(m1_2015.DateTime_sec, trend_range_DT_sec, labels=trend_labels).astype('category')

In [51]:
m1_2015.head()

,level_0,index,Date,Time,Open,High,Low,Close,Ticks,MACD_slow,MACD_fast,MACD,MACD_slow1,DateTime_sec,Trend
DateTime,,,,,,,,,,,,,,,
2015-01-02 08:00:00,0,0,2015.01.02,08:00,1.20545,1.20545,1.20514,1.20516,31,1.210236,1.209972,-0.000265,1.210024,1420185600,NaN
2015-01-02 08:01:00,1,1,2015.01.02,08:01,1.20516,1.20521,1.20503,1.20503,31,1.210115,1.209693,-0.000423,1.209765,1420185660,NaN
2015-01-02 08:02:00,2,2,2015.01.02,08:02,1.20503,1.20503,1.20468,1.20473,51,1.209985,1.209371,-0.000614,1.209504,1420185720,NaN
2015-01-02 08:03:00,3,3,2015.01.02,08:03,1.20473,1.20475,1.20346,1.20385,81,1.209825,1.208992,-0.000833,1.209210,1420185780,NaN
2015-01-02 08:04:00,4,4,2015.01.02,08:04,1.20379,1.20421,1.20375,1.20421,79,1.209657,1.208580,-0.001077,1.208950,1420185840,NaN


In [80]:
?